In [1]:
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

In [2]:
subscribers = pd.read_csv('subscribers.csv', parse_dates = ['account_creation_date'])
subscribers.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
subscribers
subscribers.dropna(subset=['attribution_survey'],inplace=True)

In [3]:
from datetime import datetime as dt

subscribers['account_creation_date'] = subscribers['account_creation_date'].dt.strftime('%Y%m')
print(subscribers['account_creation_date'])


0         202001
1         202003
2         201912
3         202001
4         201910
           ...  
227623    201911
227624    201912
227625    201912
227626    202001
227627    201911
Name: account_creation_date, Length: 224984, dtype: object


In [4]:
sub = subscribers[subscribers.payment_period>=1]
sub

,subid,package_type,num_weekly_services_utilized,preferred_genre,intended_use,weekly_consumption_hour,num_ideal_streaming_services,age,male_TF,country,...,join_fee,language,payment_type,num_trial_days,current_sub_TF,payment_period,last_payment,next_payment,cancel_date,trial_completed
0,21724479,economy,NaN,comedy,access to exclusive content,NaN,NaN,NaN,False,UAE,...,NaN,ar,Standard Charter,14,True,1,2020/2/7 21:44,2020/6/7 21:44,NaN,True
1,23383224,base,NaN,comedy,access to exclusive content,22.951579,NaN,70.0,False,UAE,...,0.3303,ar,NaN,14,True,1,2020/3/15 15:44,2020/7/15 15:44,NaN,True
4,26723159,base,4.0,comedy,replace OTT,22.951579,3.0,28.0,False,UAE,...,0.0367,ar,NaN,14,True,2,2020/2/19 12:57,2020/6/19 12:57,NaN,True
5,24810928,base,NaN,comedy,access to exclusive content,20.051667,NaN,70.0,False,UAE,...,0.3303,ar,RAKBANK,14,True,1,2020/3/17 20:15,2020/7/17 20:15,NaN,True
6,29726122,base,2.0,comedy,access to exclusive content,20.051667,2.0,61.0,False,UAE,...,0.3303,ar,Standard Charter,14,True,1,2020/3/4 18:30,2020/7/4 18:30,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227622,25835684,base,2.0,drama,access to exclusive content,24.401536,2.0,43.0,False,UAE,...,0.0000,ar,NaN,14,True,1,2020/1/15 22:43,2020/5/15 22:43,NaN,True
227623,21434712,enhanced,3.0,comedy,supplement OTT,28.751405,2.0,38.0,False,UAE,...,0.3303,ar,NaN,14,True,1,2019/12/1 14:12,2020/4/1 14:12,NaN,True
227624,25843074,enhanced,2.0,comedy,replace OTT,27.301448,2.0,49.0,False,UAE,...,0.3303,ar,Paypal,14,True,1,2019/12/20 18:02,2020/4/20 18:02,NaN,True
227625,24799085,base,NaN,comedy,access to exclusive content,31.651317,NaN,45.0,False,UAE,...,0.0367,ar,NaN,14,True,1,2020/1/4 19:40,2020/5/4 19:40,NaN,True


In [5]:
sub= sub[-((sub['cancel_date'].isnull()==False)&(sub['payment_period']==1)&(sub['trial_completed']==False))]
sub

,subid,package_type,num_weekly_services_utilized,preferred_genre,intended_use,weekly_consumption_hour,num_ideal_streaming_services,age,male_TF,country,...,join_fee,language,payment_type,num_trial_days,current_sub_TF,payment_period,last_payment,next_payment,cancel_date,trial_completed
0,21724479,economy,NaN,comedy,access to exclusive content,NaN,NaN,NaN,False,UAE,...,NaN,ar,Standard Charter,14,True,1,2020/2/7 21:44,2020/6/7 21:44,NaN,True
1,23383224,base,NaN,comedy,access to exclusive content,22.951579,NaN,70.0,False,UAE,...,0.3303,ar,NaN,14,True,1,2020/3/15 15:44,2020/7/15 15:44,NaN,True
4,26723159,base,4.0,comedy,replace OTT,22.951579,3.0,28.0,False,UAE,...,0.0367,ar,NaN,14,True,2,2020/2/19 12:57,2020/6/19 12:57,NaN,True
5,24810928,base,NaN,comedy,access to exclusive content,20.051667,NaN,70.0,False,UAE,...,0.3303,ar,RAKBANK,14,True,1,2020/3/17 20:15,2020/7/17 20:15,NaN,True
6,29726122,base,2.0,comedy,access to exclusive content,20.051667,2.0,61.0,False,UAE,...,0.3303,ar,Standard Charter,14,True,1,2020/3/4 18:30,2020/7/4 18:30,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227622,25835684,base,2.0,drama,access to exclusive content,24.401536,2.0,43.0,False,UAE,...,0.0000,ar,NaN,14,True,1,2020/1/15 22:43,2020/5/15 22:43,NaN,True
227623,21434712,enhanced,3.0,comedy,supplement OTT,28.751405,2.0,38.0,False,UAE,...,0.3303,ar,NaN,14,True,1,2019/12/1 14:12,2020/4/1 14:12,NaN,True
227624,25843074,enhanced,2.0,comedy,replace OTT,27.301448,2.0,49.0,False,UAE,...,0.3303,ar,Paypal,14,True,1,2019/12/20 18:02,2020/4/20 18:02,NaN,True
227625,24799085,base,NaN,comedy,access to exclusive content,31.651317,NaN,45.0,False,UAE,...,0.0367,ar,NaN,14,True,1,2020/1/4 19:40,2020/5/4 19:40,NaN,True


In [6]:
attributionMatrix=pd.DataFrame(columns=['bing', 'display', 'facebook','search','youtube'],index=['201907','201908','201909','201910','201911','201912','202001','202002'])

a = sub[sub['account_creation_date']=='201907']['attribution_technical'].value_counts()
attributionMatrix.at['201907','bing']  = a['bing']
attributionMatrix.at['201907','display'] = a['display']
attributionMatrix.at['201907','facebook'] = a['facebook']
attributionMatrix.at['201907','search'] = a['search']
attributionMatrix.at['201907','youtube'] = a['youtube']

a = sub[sub['account_creation_date']=='201908']['attribution_technical'].value_counts()
attributionMatrix.at['201908','bing']  = a['bing']
attributionMatrix.at['201908','display'] = a['display']
attributionMatrix.at['201908','facebook'] = a['facebook']
attributionMatrix.at['201908','search'] = a['search']
attributionMatrix.at['201908','youtube'] = a['youtube']

a = sub[sub['account_creation_date']=='201909']['attribution_technical'].value_counts()
attributionMatrix.at['201909','bing']  = a['bing']
attributionMatrix.at['201909','display'] = a['display']
attributionMatrix.at['201909','facebook'] = a['facebook']
attributionMatrix.at['201909','search'] = a['search']
attributionMatrix.at['201909','youtube'] = a['youtube']

a = sub[sub['account_creation_date']=='201910']['attribution_technical'].value_counts()
attributionMatrix.at['201910','bing']  = a['bing']
attributionMatrix.at['201910','display'] = a['display']
attributionMatrix.at['201910','facebook'] = a['facebook']
attributionMatrix.at['201910','search'] = a['search']
attributionMatrix.at['201910','youtube'] = a['youtube']

a = sub[sub['account_creation_date']=='201911']['attribution_technical'].value_counts()
attributionMatrix.at['201911','bing']  = a['bing']
attributionMatrix.at['201911','display'] = a['display']
attributionMatrix.at['201911','facebook'] = a['facebook']
attributionMatrix.at['201911','search'] = a['search']
attributionMatrix.at['201911','youtube'] = a['youtube']

a = sub[sub['account_creation_date']=='201912']['attribution_technical'].value_counts()
attributionMatrix.at['201912','bing']  = a['bing']
attributionMatrix.at['201912','display'] = a['display']
attributionMatrix.at['201912','facebook'] = a['facebook']
attributionMatrix.at['201912','search'] = a['search']
attributionMatrix.at['201912','youtube'] = a['youtube']

a = sub[sub['account_creation_date']=='202001']['attribution_technical'].value_counts()
attributionMatrix.at['202001','bing']  = a['bing']
attributionMatrix.at['202001','display'] = a['display']
attributionMatrix.at['202001','facebook'] = a['facebook']
attributionMatrix.at['202001','search'] = a['search']
attributionMatrix.at['202001','youtube'] = a['youtube']

a = sub[sub['account_creation_date']=='202002']['attribution_technical'].value_counts()
attributionMatrix.at['202002','bing']  = a['bing']
attributionMatrix.at['202002','display'] = a['display']
attributionMatrix.at['202002','facebook'] = a['facebook']
attributionMatrix.at['202002','search'] = a['search']
attributionMatrix.at['202002','youtube'] = a['youtube']

In [7]:
attributionMatrix

,bing,display,facebook,search,youtube
201907,24,7,2004,443,1
201908,30,33,4809,1165,11
201909,51,8,3726,1294,1
201910,53,28,6264,1673,7
201911,59,64,8459,1856,36
201912,13,65,6026,2688,55
202001,93,325,4096,2846,312
202002,179,189,4893,2944,159


In [8]:
spending = pd.read_csv('channel_spend_graduate .csv')
spending

,channel,date,spend_AED
0,bing,201907,400
1,display,201907,12
2,facebook,201907,9000
3,search,201907,13000
4,youtube,201907,90
5,bing,201908,900
6,display,201908,20
7,facebook,201908,13000
8,search,201908,18500
9,youtube,201908,180


In [9]:
items = {'bing' : pd.Series(data = [400,900,1000,1100,1300,300,2100,3700], index = ['201907','201908','201909','201910','201911','201912','202001','202002']),
         'display' : pd.Series(data = [12, 20, 13,19,29,31,148,94], index = ['201907','201908','201909','201910','201911','201912','202001','202002']),
         'facebook': pd.Series(data = [9000,13000,10500,17000,23000,16000,11000,14000], index = ['201907','201908','201909','201910','201911','201912','202001','202002']),
         'search': pd.Series(data = [13000,18500,19000,24000,25000,38000,41000,44000], index = ['201907','201908','201909','201910','201911','201912','202001','202002']),
        'youtube': pd.Series(data = [90, 180, 100,130,550,900,4360,2420], index = ['201907','201908','201909','201910','201911','201912','202001','202002'])}
                              
matrix = pd.DataFrame(items)
matrix


,bing,display,facebook,search,youtube
201907,400,12,9000,13000,90
201908,900,20,13000,18500,180
201909,1000,13,10500,19000,100
201910,1100,19,17000,24000,130
201911,1300,29,23000,25000,550
201912,300,31,16000,38000,900
202001,2100,148,11000,41000,4360
202002,3700,94,14000,44000,2420


In [10]:
CAC_last_interaction = matrix.div(attributionMatrix,axis=1)

In [11]:
CAC_last_interaction

,bing,display,facebook,search,youtube
201907,16.6667,1.71429,4.49102,29.3454,90
201908,30,0.606061,2.70326,15.8798,16.3636
201909,19.6078,1.625,2.81804,14.6832,100
201910,20.7547,0.678571,2.71392,14.3455,18.5714
201911,22.0339,0.453125,2.719,13.4698,15.2778
201912,23.0769,0.476923,2.65516,14.1369,16.3636
202001,22.5806,0.455385,2.68555,14.4062,13.9744
202002,20.6704,0.497354,2.86123,14.9457,15.2201


In [12]:
aMatrix=pd.DataFrame(columns=['facebook','search','youtube'],index=['201907','201908','201909','201910','201911','201912','202001','202002'])

b = sub[sub['account_creation_date']=='201907']['attribution_survey'].value_counts()


aMatrix.at['201907','facebook'] = b['facebook']
aMatrix.at['201907','search'] = b['search']
aMatrix.at['201907','youtube'] = b['youtube']

b = sub[sub['account_creation_date']=='201908']['attribution_survey'].value_counts()


aMatrix.at['201908','facebook'] = b['facebook']
aMatrix.at['201908','search'] = b['search']
aMatrix.at['201908','youtube'] = b['youtube']

b = sub[sub['account_creation_date']=='201909']['attribution_survey'].value_counts()


aMatrix.at['201909','facebook'] = b['facebook']
aMatrix.at['201909','search'] = b['search']
aMatrix.at['201909','youtube'] = b['youtube']

b = sub[sub['account_creation_date']=='201910']['attribution_survey'].value_counts()


aMatrix.at['201910','facebook'] = b['facebook']
aMatrix.at['201910','search'] = b['search']
aMatrix.at['201910','youtube'] = b['youtube']

b = sub[sub['account_creation_date']=='201911']['attribution_survey'].value_counts()


aMatrix.at['201911','facebook'] = b['facebook']
aMatrix.at['201911','search'] = b['search']
aMatrix.at['201911','youtube'] = b['youtube']

b = sub[sub['account_creation_date']=='201912']['attribution_survey'].value_counts()

aMatrix.at['201912','facebook'] = b['facebook']
aMatrix.at['201912','search'] = b['search']
aMatrix.at['201912','youtube'] = b['youtube']

b = sub[sub['account_creation_date']=='202001']['attribution_survey'].value_counts()

aMatrix.at['202001','facebook'] = b['facebook']
aMatrix.at['202001','search'] = b['search']
aMatrix.at['202001','youtube'] = b['youtube']

b = sub[sub['account_creation_date']=='202002']['attribution_survey'].value_counts()

aMatrix.at['202002','facebook'] = b['facebook']
aMatrix.at['202002','search'] = b['search']
aMatrix.at['202002','youtube'] = b['youtube']

aMatrix

,facebook,search,youtube
201907,2754,282,40
201908,7508,544,204
201909,6939,633,166
201910,8620,664,218
201911,10816,678,252
201912,8568,832,200
202001,6451,924,630
202002,7263,1063,462


In [13]:
item = {
         'facebook': pd.Series(data = [9000,13000,10500,17000,23000,16000,11000,14000], index = ['201907','201908','201909','201910','201911','201912','202001','202002']),
         'search': pd.Series(data = [13000,18500,19000,24000,25000,38000,41000,44000], index = ['201907','201908','201909','201910','201911','201912','202001','202002']),
        'youtube': pd.Series(data = [90, 180, 100,130,550,900,4360,2420], index = ['201907','201908','201909','201910','201911','201912','202001','202002'])}
                              
cost = pd.DataFrame(item)
cost

,facebook,search,youtube
201907,9000,13000,90
201908,13000,18500,180
201909,10500,19000,100
201910,17000,24000,130
201911,23000,25000,550
201912,16000,38000,900
202001,11000,41000,4360
202002,14000,44000,2420


In [14]:
CAC_survey = cost.div(aMatrix,axis=1)
CAC_survey

,facebook,search,youtube
201907,3.26797,46.0993,2.25
201908,1.73149,34.0074,0.882353
201909,1.51319,30.0158,0.60241
201910,1.97216,36.1446,0.59633
201911,2.12648,36.8732,2.18254
201912,1.86741,45.6731,4.5
202001,1.70516,44.3723,6.92063
202002,1.92758,41.3923,5.2381
